<a href="https://colab.research.google.com/github/moonsejin315/Python/blob/main/LLM_%EC%9D%91%EC%9A%A9_PDF_%EC%9A%94%EC%95%BD_%EC%9B%B9%EC%82%AC%EC%9D%B4%ED%8A%B8_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install PyPDF2
!pip install sentence-transformers
!pip install gradio
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64

In [2]:
!pip install faiss-cpu
!pip install openapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 27.9 MB/s eta 0:00:00


In [4]:
import os

from PyPDF2 import PdfReader

import gradio as gr

#텍스트 전처리 관련 라이브러리
#문저를 '청크화'하는데 쓰이는 라이브러리
from langchain.text_splitter import CharacterTextSplitter
#청크화된 문서를 '임베딩'하는 데 쓰이는 라이브러리
from langchain.embeddings import HuggingFaceEmbeddings
#벡터스토어, 벡터db와 관련된 라이브러리
from langchain import FAISS

#LLM과의 통신
#질의 응답 체인
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
#openai에서 보내는 콜백을 다루는 라이브러리
from langchain.callbacks import get_openai_callback

In [26]:
#텍스트 전처리 수행하는 함수

def preprocess_text(text):

  #characterTextSplitter를 써서 청크로 분할
  text_splitter = CharacterTextSplitter(separator='\n',
                                        chunk_size=1000,
                                        chunk_overlap=200,
                                        length_function=len)
  #위에 정의한 text_splitter로 text를 청크로 만드는 실제 작업
  chunk = text_splitter.split_text(text)

  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
  documents = FAISS.from_texts(chunk, embeddings)

  return documents

In [31]:
def summary(pdf):
  os.environ['OPENAI_API_KEY'] = '###############################'

  pdf_reader = PdfReader(pdf)
  text = ""

  #for 페이지 in pdf_reader가 읽어오는 콘텐츠
  for page in pdf_reader.pages:
    text += page.extract_text()

    #LLM 사용을 위한 임베딩-벡터화를 거친 text를 documents의 형태로 받다.
    documents = preprocess_text(text)
    query = "업로드된 PDF 파일의 내용을 3~5문장으로 요약하여 주세요. 영어 pdf여도 한글로 요약해주세요."

    if query:
      docs = documents.similarity_search(query)

      llm = ChatOpenAI(model='gpt-4o', temperature=0)
      chain = load_qa_chain(llm, chain_type='stuff')

      with get_openai_callback() as cost:
        response = chain.run(input_documents=docs, question=query)

        return response

In [13]:
import os
from pathlib import Path

In [14]:
def upload_file(filepath):
  name = Path(filepath).name
  return gr.UploadButton(visible=False)

In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [32]:
result = summary('/content/[K-DATA] 빅데이터분석기사 실기 체험환경 가이드.pdf')
print(result)

이 문서는 빅데이터분석기사 실기 시험의 체험 환경 가이드입니다. 수험자가 사전에 시험 환경을 체험할 수 있도록 예시 문제를 제공하며, 클라우드 기반의 크롬 브라우저를 사용하여 시험을 진행합니다. 시험 중에는 제공된 패키지만 사용할 수 있고, 코드 실행 시간은 1분으로 제한됩니다. 또한, 코드 라인별 실행, 그래프 기능, 단축키 자동완성 기능은 제공되지 않습니다.


In [34]:
with gr.Blocks() as demo:
    upload_file = gr.File(label="Upload PDF")
    output = gr.Textbox(label="Output")

    upload_file.change(summary, inputs=upload_file, outputs=output)

# 인터페이스 실행
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://cae63154b4ea366e9a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
